# Taller 4: Implementar en Notebook Jupyter una Sistema de línea de espera con un servidor (Modelo de colas G/G/1)

Diego Fernando Palacios Hoyos
Codigo: 160003727

In [20]:
from datetime import datetime
import math as m
import numpy as np


def mixtoU():
    global U
    a=22695477
    c=1
    m=2**32
    Xo=U
    Xn=(a*Xo+c)%m
    ui=Xn/m
    U=Xn
    return ui

def mixtoU2():
    global U2
    a=65539
    c=0
    m=2**31
    Xo=U2
    Xn=(a*Xo+c)%m
    ui=Xn/m
    U2=Xn
    return ui

def exponencial(lamda):
    if lamda==A:
        U=mixtoU()
        return (-1/lamda)*m.log(1-U)
    if lamda==B:
        U=mixtoU2()
        return (-1/lamda)*m.log(1-U)

def llegada(tsuc):
    global n,Nll,Tsuc_tLL,Tsuc_ts,t,Ns,nmax
    t=tsuc
    n=n+1
    if n>nmax:
        nmax=n
    Nll=Nll+1
    LL[Nll]=t
    X=exponencial(A) # generar F
    if t+X<T:
        Tsuc_tLL=t+X
    if n==1:
        Y=exponencial(B)
        Tsuc_ts=t+Y
        Serv[Ns+1]=Y

def servidor(tsuc):
    global t,n,Tsuc_tLL,Tsuc_ts,Ns
    t=tsuc
    n=n-1
    Ns=Ns+1
    S[Ns]=t
    if n>0:
        Y= exponencial(B) # generar G
        Tsuc_ts=t+Y
        Serv[Ns]=Y

In [21]:
U = datetime.now().microsecond
U2 = 99999*datetime.now().microsecond

A=3
B=5

t = tsuc = Nll = Ns = n = 0
M=2**16
Tsuc_tLL=M
Tsuc_ts=M

LL=np.zeros(1000)
S=np.zeros(1000)
Serv=np.zeros(1000)
T=100
nmax=0

X=exponencial(A)
if(X>T):
    Tp=0
    t_med_sistema=0
    t_med_cola=0
else:
    llegada(X)
    while(Tsuc_tLL!=M or Tsuc_ts!=M):
        if Tsuc_tLL<Tsuc_ts:
            tsuc=Tsuc_tLL
            Tsuc_tLL=M
            llegada(tsuc)
        if Tsuc_ts<Tsuc_tLL:
            tsuc=Tsuc_ts
            Tsuc_ts=M
            servidor(tsuc)
    Tp=max(0,t-T)
    acumulado1= acumulado2 = ind = 0
    while(ind<Nll):
        acumulado1=acumulado1+S[ind]-LL[ind]
        acumulado2=acumulado2+S[ind]-LL[ind]-Serv[ind]
        ind=ind+1
    t_med_sistema=acumulado1/Nll
    t_med_cola=acumulado2/Nll

print("Tiempo promedio de clientes en el sistema: ",t_med_sistema)
print("Tiempo promedio de clientes en la cola: ",t_med_cola)
print("Tiempo transcurrido desde  T  hasta que el ultimo cliente abandona el sistema: ",Tp)
print("Numero maximo de clientes en el sistema durante la simulacion: ",nmax)

Tiempo promedio de clientes en el sistema:  0.39304464234333997
Tiempo promedio de clientes en la cola:  0.25641142099961317
Tiempo transcurrido desde  T  hasta que el ultimo cliente abandona el sistema:  0.7286082815173671
Numero maximo de clientes en el sistema durante la simulacion:  9
